In [36]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

# preprocessing
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
# import pandas_profiling as pp


# NN models
import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras import optimizers
from keras.wrappers.scikit_learn import KerasClassifier
from keras.callbacks import EarlyStopping, ModelCheckpoint

# import warnings filter
from warnings import simplefilter
# ignore all future warnings
simplefilter(action='ignore', category=FutureWarning)

In [37]:
data = pd.read_csv("cardio_train.csv", sep=";")

In [38]:

data.drop("id",axis=1,inplace=True)
data.drop_duplicates(inplace=True)

# feature extraction
data["bmi"] = data["weight"] / (data["height"]/100)**2
out_filter = ((data["ap_hi"]>250) | (data["ap_lo"]>200))
data = data[~out_filter]
len(data)

68983

In [39]:
# filtering
out_filter2 = ((data["ap_hi"] < 0) | (data["ap_lo"] < 0))
data = data[~out_filter2]

In [40]:
data.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68975 entries, 0 to 69999
Data columns (total 13 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          68975 non-null  int64  
 1   gender       68975 non-null  int64  
 2   height       68975 non-null  int64  
 3   weight       68975 non-null  float64
 4   ap_hi        68975 non-null  int64  
 5   ap_lo        68975 non-null  int64  
 6   cholesterol  68975 non-null  int64  
 7   gluc         68975 non-null  int64  
 8   smoke        68975 non-null  int64  
 9   alco         68975 non-null  int64  
 10  active       68975 non-null  int64  
 11  cardio       68975 non-null  int64  
 12  bmi          68975 non-null  float64
dtypes: float64(2), int64(11)
memory usage: 7.4 MB


In [41]:
target_name = 'cardio'
data_target = data[target_name]
data = data.drop([target_name], axis=1)

In [42]:
train, test, target, target_test = train_test_split(data, data_target, test_size=0.2, random_state=0)

In [43]:
train.info()


<class 'pandas.core.frame.DataFrame'>
Int64Index: 55180 entries, 31684 to 69281
Data columns (total 12 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   age          55180 non-null  int64  
 1   gender       55180 non-null  int64  
 2   height       55180 non-null  int64  
 3   weight       55180 non-null  float64
 4   ap_hi        55180 non-null  int64  
 5   ap_lo        55180 non-null  int64  
 6   cholesterol  55180 non-null  int64  
 7   gluc         55180 non-null  int64  
 8   smoke        55180 non-null  int64  
 9   alco         55180 non-null  int64  
 10  active       55180 non-null  int64  
 11  bmi          55180 non-null  float64
dtypes: float64(2), int64(10)
memory usage: 5.5 MB


In [44]:
#%% split training set to validation set
Xtrain, Xval, Ztrain, Zval = train_test_split(train, target, test_size=0.2, random_state=0)


In [45]:
def build_ann(optimizer='adam'):
    
    # Initializing the ANN
    ann = Sequential()
    
    # Adding the input layer and the first hidden layer of the ANN with dropout
    ann.add(Dense(units=32, kernel_initializer='glorot_uniform', activation='relu', input_shape=(len(train.columns),)))
    
    # Add other layers, it is not necessary to pass the shape because there is a layer before
    ann.add(Dense(units=64, kernel_initializer='glorot_uniform', activation='relu'))
    ann.add(Dropout(rate=0.5))
    ann.add(Dense(units=64, kernel_initializer='glorot_uniform', activation='relu'))
    ann.add(Dropout(rate=0.5))
    
    # Adding the output layer
    ann.add(Dense(units=1, kernel_initializer='glorot_uniform', activation='sigmoid'))
    
    # Compiling the ANN
    ann.compile(optimizer=optimizer, loss='binary_crossentropy', metrics=['accuracy'])
    
    return ann

In [47]:
opt = optimizers.Adam(lr=0.001)
ann = build_ann(opt)
# Training the ANN
history = ann.fit(Xtrain, Ztrain, batch_size=16, epochs=20, validation_data=(Xval, Zval))

c:\Python310\lib\site-packages\keras\optimizers\legacy\adam.py:117: UserWarning: The `lr` argument is deprecated, use `learning_rate` instead.
  super().__init__(name, **kwargs)


Epoch 1/20
2759/2759 [==============================] - 9s 3ms/step - loss: 41.1823 - accuracy: 0.5057 - val_loss: 0.6930 - val_accuracy: 0.5097
Epoch 2/20
2759/2759 [==============================] - 8s 3ms/step - loss: 0.6931 - accuracy: 0.5059 - val_loss: 0.6930 - val_accuracy: 0.5097
Epoch 3/20
2759/2759 [==============================] - 8s 3ms/step - loss: 0.6931 - accuracy: 0.5052 - val_loss: 0.6930 - val_accuracy: 0.5097
Epoch 4/20
2759/2759 [==============================] - 8s 3ms/step - loss: 0.6931 - accuracy: 0.5046 - val_loss: 0.6930 - val_accuracy: 0.5097
Epoch 5/20
2759/2759 [==============================] - 8s 3ms/step - loss: 0.6931 - accuracy: 0.5059 - val_loss: 0.6931 - val_accuracy: 0.5097
Epoch 6/20
2759/2759 [==============================] - 8s 3ms/step - loss: 0.6931 - accuracy: 0.5059 - val_loss: 0.6930 - val_accuracy: 0.5097
Epoch 7/20
2759/2759 [==============================] - 8s 3ms/step - loss: 0.6931 - accuracy: 0.5044 - val_loss: 0.6930 - val_accuracy

In [48]:
# Predicting the Train set results
ann_prediction = ann.predict(train)
ann_prediction = (ann_prediction > 0.5)*1 # convert probabilities to binary output
from sklearn.metrics import accuracy_score
# Compute error between predicted data and true response and display it in confusion matrix
acc_ann1 = round(accuracy_score(target, ann_prediction) * 100, 2)
acc_ann1

1725/1725 [==============================] - 3s 2ms/step


50.67

In [49]:


# Predicting the Test set results
ann_prediction_test = ann.predict(test)
ann_prediction_test = (ann_prediction_test > 0.5)*1 # convert probabilities to binary output

# Compute error between predicted data and true response and display it in confusion matrix
acc_test_ann1 = round(accuracy_score(target_test, ann_prediction_test) * 100, 2)
acc_test_ann1

432/432 [==============================] - 1s 2ms/step


49.85

In [54]:
single_inp = [[50, 1, 150, 75, 120, 90, 1, 1, 1, 1, 1, 24.62300]]
out = ann.predict(single_inp)
out

1/1 [==============================] - 0s 219ms/step


array([[0.0094826]], dtype=float32)